In [13]:
from sim import simulation as sim
import dask
import numpy as np
import xarray as xr
from netCDF4 import Dataset

In [14]:
point = sim.SystemState([1,2,3,4,5,6], 0)
traj = sim.Simulator(system_state=point)

In [15]:
traj


system model: <function lorenz_96 at 0x7f75c1382620>
integration method: <function runge_kutta_4 at 0x7f75d03da1e0>
time increment: 0.01
forcing: 0

--- system state ---
coordinates: [1, 2, 3, 4, 5, 6]
time: 0

In [16]:
integration_time = 2
integration_steps = int(integration_time/traj.increment)
chunk_steps = 100
chunks = int(integration_steps/chunk_steps)

In [17]:
chunks

2

In [18]:
#last_coords = point.coords
#last_time = point.time
#last_coords

In [19]:
#data_array = xr.DataArray(last_coords, dims=('time', 'node'), coords={'time': 0, 'node': [0, 1, 2, 3, 4, 5]})
#dim = len(point.coords)
#np.empty((chunk_steps, dim))

In [20]:
Dataset('/home/cucchi/phd/prova.nc', 'w',  format='NETCDF4_CLASSIC')

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    dimensions(sizes): 
    variables(dimensions): 
    groups: 

In [21]:
dim = len(point.coords)

for chunk in np.arange(0, chunks):
    # initialize chunk numpy array
    print(chunk)
    t = []
    data_nparray = np.empty((chunk_steps, dim))
    for i in np.arange(0, chunk_steps):
        data_nparray[i,:] = point.coords
        t.append(point.time)
        traj.integrate()
    data_array = xr.DataArray(data_nparray, dims=('time', 'node'), coords={'time': t, 'node': [0, 1, 2, 3, 4, 5]})
    print(data_array)
    data_array.to_netcdf('/home/cucchi/phd/prova.nc', mode='a')

0
<xarray.DataArray (time: 100, node: 6)>
array([[ 1.      ,  2.      ,  3.      ,  4.      ,  5.      ,  6.      ],
       [ 0.810812,  1.954149,  3.031507,  4.051934,  5.065492,  5.783991],
       [ 0.624461,  1.915952,  3.066053,  4.107472,  5.121518,  5.555994],
       ...,
       [ 0.714564, -0.128924,  2.340752, -1.022269, -2.317817,  0.808338],
       [ 0.724428, -0.116571,  2.319613, -1.062538, -2.278715,  0.759999],
       [ 0.732952, -0.104002,  2.298519, -1.101331, -2.239102,  0.711736]])
Coordinates:
  * time     (time) float64 0.0 0.01 0.02 0.03 0.04 ... 0.95 0.96 0.97 0.98 0.99
  * node     (node) int64 0 1 2 3 4 5
1
<xarray.DataArray (time: 100, node: 6)>
array([[ 0.740176, -0.091237,  2.277452, -1.138669, -2.198985,  0.663657],
       [ 0.74614 , -0.078294,  2.256393, -1.17457 , -2.158375,  0.615865],
       [ 0.750889, -0.065197,  2.235327, -1.209053, -2.117284,  0.568459],
       ...,
       [ 0.233941,  0.374419,  0.505399, -1.043299,  0.453022, -0.160437],
       [ 

In [22]:
traj


system model: <function lorenz_96 at 0x7f75c1382620>
integration method: <function runge_kutta_4 at 0x7f75d03da1e0>
time increment: 0.01
forcing: 0

--- system state ---
coordinates: [ 0.2274022   0.36771829  0.47666793 -1.01122999  0.45907396 -0.13873612]
time: 2.0000000000000013

In [23]:
data = xr.open_dataarray('/home/cucchi/phd/prova.nc')

In [24]:
data

<xarray.DataArray (time: 100, node: 6)>
array([[ 1.      ,  2.      ,  3.      ,  4.      ,  5.      ,  6.      ],
       [ 0.810812,  1.954149,  3.031507,  4.051934,  5.065492,  5.783991],
       [ 0.624461,  1.915952,  3.066053,  4.107472,  5.121518,  5.555994],
       ...,
       [ 0.714564, -0.128924,  2.340752, -1.022269, -2.317817,  0.808338],
       [ 0.724428, -0.116571,  2.319613, -1.062538, -2.278715,  0.759999],
       [ 0.732952, -0.104002,  2.298519, -1.101331, -2.239102,  0.711736]])
Coordinates:
  * time     (time) float64 0.0 0.01 0.02 0.03 0.04 ... 0.95 0.96 0.97 0.98 0.99
  * node     (node) int64 0 1 2 3 4 5

In [ ]:
%time sim.lorenz_96([1,2,3,4], 1)

In [ ]:
@dask.delayed
def _add(a, b):
    return a + b

@dask.delayed
def _multiply(a, b):
    return a * b

@dask.delayed
def _square(a):
    return a**2

def _lorenz_96_1d(x: list, i: int, n:int, f:float) -> list:
    
    d = (x[(i+1) % n] - x[i-2]) * x[i-1] - x[i] + f
    
    return d

def lorenz_96_delayed(x, forcing):

    """
    This method implement Lorenz-96 first-order differential equations system.
    :param x: coordinates at time t0
    :param forcing: (constant) forcing term
    :return: state derivatives at time t0
    """
    n = len(x)

    d = []

    for i in range(0, n):
        dx = _lorenz_96_1d(x, i, n, forcing)
        d.append(dx)

    return d

In [ ]:
a = []
d = []
x = [i for i in range(1,10)]
n = len(x)
forcing = 1

for t in range(0, 5):
    x1 = []
    for i in range(0,n):
        dx = dask.delayed(_lorenz_96_1d)(x, i, n, forcing)
        x_new = _add(x[i], dx)
        #a = _square(i)
        x1.append(x_new)
    a.append(dask.delayed(sum)(x1))
    x = x1
    
d = dask.delayed(sum)(a)

In [ ]:
%time d.compute()

In [ ]:
a = []
d = []
x = [i for i in range(1,10)]
n = len(x)
forcing = 1

for t in range(0, 5):
    x1 = []
    for i in range(0,n):
        dx = _lorenz_96_1d(x, i, n, forcing)
        x_new = x[i] + dx
        #a = _square(i)
        x1.append(x_new)
    a.append(sum(x1))
    x = x1
    
%time d = sum(a)

In [ ]:
d

In [ ]:
def inc(x):
    return x + 1

def double(x):
    return x + 2

def add(x, y):
    return x + y

data = [1, 2, 3, 4, 5]

output = []
for x in data:
    a = dask.delayed(inc)(x)
    b = dask.delayed(double)(x)
    c = dask.delayed(add)(a, b)
    output.append(c)

total = dask.delayed(sum)(output)

In [ ]:
total.visualize()

In [ ]:
def inc(x):
    return x + 1

def double(x):
    return x + 2

def add(x, y):
    return x + y

data = [1, 2, 3, 4, 5]

output = []
for x in data:
    a = inc(x)
    b = double(x)
    c = add(a, b)
    output.append(c)

%time total = sum(output)

In [ ]:
output = []
for x in data:
    a = dask.delayed(inc)(x)
    b = dask.delayed(double)(x)
    c = dask.delayed(add)(a, b)
    output.append(c)

total = dask.delayed(sum)(output)

%time total.compute()